In [ ]:
import os
import cv2
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

import timm
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data.sampler import Sampler
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau

import torchmetrics
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping


import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, precision_score, recall_score

import warnings
warnings.filterwarnings(action='ignore')

N_EPOCHS = 1000
BATCH_SIZE = 256
LEARNING_RATE = 0.0005
PAITENCE = 10

IM_HEIGHT = 256
IM_WIDTH = 256

classification_mode = "LVI_Positive-Negative"
model_name = "resnet50"

In [ ]:
def generate_patch_df(flist, label):
    df = pd.DataFrame({"fpath": flist})
    df['slide_id'] = df['fpath'].map(lambda x: x.split("/")[-1].split(".")[0].split("_")[0])
    df['patient_id'] = df['slide_id'].map(lambda x: x.split("-")[0])
    df['target'] = label

    df = df.loc[:, ["patient_id", "slide_id", "fpath", "target"]]
    
    return df


def define_dataset(positive_df, negative_df, sampling_rate=0.2):
    X_train_positive, X_test_positive, y_train_positive, y_test_positive = train_test_split(positive_df['fpath'], positive_df['target'], test_size=sampling_rate, random_state=1234)
    X_train_positive, X_valid_positive, y_train_positive, y_valid_positive = train_test_split(X_train_positive, y_train_positive, test_size=sampling_rate, random_state=1234)

    X_train_negative, X_test_negative, y_train_negative, y_test_negative = train_test_split(negative_df['fpath'], negative_df['target'], test_size=sampling_rate, random_state=1234)
    X_train_negative, X_valid_negative, y_train_negative, y_valid_negative = train_test_split(X_train_negative, y_train_negative, test_size=sampling_rate, random_state=1234)
    
    X_train = np.hstack([X_train_positive, X_train_negative])
    X_valid = np.hstack([X_valid_positive, X_valid_negative])
    X_test = np.hstack([X_test_positive, X_test_negative])

    y_train = np.hstack([y_train_positive, y_train_negative])
    y_valid = np.hstack([y_valid_positive, y_valid_negative])
    y_test = np.hstack([y_test_positive, y_test_negative])
    
    return X_train, X_valid, X_test, y_train, y_valid, y_test
    
    
positive_flist = glob.glob("data/LVI_dataset/patch_image_size-300_overlap-0/LVI/*.png")
negative_flist = glob.glob("data/LVI_dataset/patch_image_size-300_overlap-0/Negative/*.png")

positive_flist = generate_patch_df(positive_flist, 1)
negative_flist = generate_patch_df(normal_flist, 0)

X_train, X_valid, X_test, y_train, y_valid, y_test = define_dataset(positive_flist, negative_flist, sampling_rate=0.2)
print(f"X train: {X_train.shape}\nX valid: {X_valid.shape}\nX test: {X_test.shape}")
print(f"y train: {y_train.shape}\ny valid: {y_valid.shape}\ny test: {y_test.shape}")


In [ ]:
train_transforms = A.Compose([ 

    A.RandomCrop(width=IM_WIDTH, height=IM_HEIGHT, p=1.0),
    
    A.OneOf([
        A.Transpose(),
        A.HorizontalFlip(),
        A.VerticalFlip()
    ], p=0.5),

    A.OneOf([
       A.ElasticTransform(),
       A.Rotate(25)
    ], p=0.8),

    A.OneOf([
       A.Blur(),
       A.GaussianBlur(),
       A.GaussNoise(),
       A.MedianBlur()
    ], p=0.5),

    A.OneOf([
       A.ChannelShuffle(),
       A.ColorJitter(),
       A.HueSaturationValue(),
       A.RandomBrightnessContrast()
    ], p=0.5),
    
    A.Normalize(p=1.0),
    ToTensorV2()
])


valid_transforms = A.Compose([ 
    A.Resize(width=IM_WIDTH, height=IM_HEIGHT, p=1.0),
    A.Normalize(p=1.0),
    ToTensorV2()
])


class LVIDataset(Dataset):
    def __init__(self, X, y, transforms):
        self.X = X
        self.y = y
        self.transforms = transforms
        
    def __len__(self):
        return self.X.shape[0]
    
    def __getitem__(self, idx):
        image  = cv2.imread(self.X[idx])
        target = self.y[idx]

        augmented = self.transforms(image=image)
        image = augmented['image']
        
        return image, target

    
train_dataset = LVIDataset(X_train, y_train, transforms=train_transforms)
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, num_workers=12, pin_memory=True, shuffle=True)

valid_dataset = LVIDataset(X_valid, y_valid, transforms=valid_transforms)
valid_dataloader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, num_workers=12, pin_memory=True, shuffle=True)

test_dataset = LVIDataset(X_test, y_test, transforms=valid_transforms)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, num_workers=12, pin_memory=True, shuffle=False)

In [ ]:
class ImageClassifier(pl.LightningModule):
    def __init__(self, model_name, learning_rate, num_classes=3):
        super(ImageClassifier, self).__init__()
        self.model = timm.create_model(model_name, num_classes=num_classes, pretrained=True)
        self.learning_rate = learning_rate
        
        self.train_accuracy = torchmetrics.Accuracy()
        self.valid_accuracy = torchmetrics.Accuracy()
        self.test_accuracy = torchmetrics.Accuracy()
        
        self.train_f1 = torchmetrics.F1()
        self.valid_f1 = torchmetrics.F1()
        self.test_f1 = torchmetrics.F1()
    
    
    def forward(self, x):
        pred = self.model(x)

        return pred


    def predict_step(self, batch, batch_idx, dataloader_idx=0):
        x, y = batch
        y_hat = self.model(x)
        
        return y_hat, y
    
    
    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.model(x)
        loss = F.cross_entropy(y_hat, y)
        self.log("train_loss", loss)
        self.log("train_accuracy", self.train_accuracy(y_hat, y), on_step=True, on_epoch=True, prog_bar=True, logger=True)
        self.log("train_f1", self.train_f1(y_hat, y), on_step=True, on_epoch=True, prog_bar=True, logger=True)

        return loss
    
        
    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.model(x)
        loss = F.cross_entropy(y_hat, y)
        self.log("valid_loss", loss)
        self.log("valid_accuracy", self.valid_accuracy(y_hat, y), on_step=False, on_epoch=True, prog_bar=True, logger=True)
        self.log("valid_f1", self.valid_f1(y_hat, y), on_step=False, on_epoch=True, prog_bar=True, logger=True)

    
    def test_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.model(x)
        loss = F.cross_entropy(y_hat, y)
        self.log("test_loss", loss)
        self.log("test_accuracy", self.test_accuracy(y_hat, y), on_step=False, on_epoch=True, prog_bar=True, logger=True)
        self.log("test_f1", self.test_f1(y_hat, y), on_step=False, on_epoch=True, prog_bar=True, logger=True)
          
        
    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.parameters(), lr=self.learning_rate)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="min", factor=0.2, patience=20, min_lr=5e-5)
        
        return {"optimizer": optimizer, "lr_scheduler": scheduler, "monitor": "valid_loss"}

    
def define_callbacks(patience):
    return EarlyStopping('valid_loss', patience=patience)

In [ ]:
model = ImageClassifier(model_name, LEARNING_RATE)
callbacks = define_callbacks(PAITENCE)
# trainer = pl.Trainer(accelerator="cpu", num_processes=1, max_epochs=1, enable_progress_bar=True)
trainer = pl.Trainer(gpus=1, max_epochs=N_EPOCHS, enable_progress_bar=True, callbacks=callbacks)
trainer.fit(model, train_dataloader, valid_dataloader)

In [ ]:
torch.save(model.state_dict(), f"./weights/{classification_mode}_{model_name}_patch-{IM_WIDTH}.pt")

In [ ]:
res = trainer.predict(model, dataloaders=test_dataloader)
pred = np.argmax(res[0][0], axis=1)
target = res[0][1]

confusion_matrix(pred, target)